# 1. Design a layer that takes an input and computes a tensor reduction, i.e., it returns $y_k=\sum_{i,j}W_{i,j,k}x_ix_j$.



In [1]:
import torch
a = torch.ones(3,2)
chunks = torch.chunk(a,3,dim=0)
chunks[0]

tensor([[1., 1.]])

In [4]:
b = torch.ones(1,3)
c = torch.matmul(b.T,b)
c

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [12]:
import torch.nn as nn
import torch

class ReductionLayer(nn.Module):
    def __init__(self, num_inputs, k):
        super().__init__()
        self.w = nn.ParameterList([nn.Parameter(torch.randn(num_inputs, num_inputs)) for i in range(k)])
        
    def stat_row(self, X):
        y = []
        for part_w in self.w:
            y.append((part_w*X).sum())
        print(y)
        return torch.cat(y.reshape(-1,1),dim=-1)
        
    def forward(self, X):
        chunks = torch.chunk(X,X.shape[0],dim=0)
        rows = []
        for row in chunks:
            row = row.reshape(1,-1)
            part_x = torch.matmul(row.T,row)
            rows.append(self.stat_row(part_x))
        return torch.cat(rows,dim=0)

In [13]:
layer = ReductionLayer(5,2)
x = torch.randn(1,5)
layer(x)

[tensor(2.8652, grad_fn=<SumBackward0>), tensor(3.3701, grad_fn=<SumBackward0>)]


AttributeError: 'list' object has no attribute 'reshape'

# 2. Design a layer that returns the leading half of the Fourier coefficients of the data.